In [1]:
using DataFrames, XLSX, DataFramesMeta
using LinearAlgebra
using LightGraphs, SimpleWeightedGraphs

# Network Data

In [18]:
Bus = DataFrame(XLSX.readtable("All-Ireland System- WinterPeak 2026 - PSSE Data.xlsx", "BusData", infer_eltypes=true)... );

In [17]:
BusDist =  @linq Bus |>
            where(:kV .<= 38) |>
            select(:Bus,:kV);
size(BusDist)

(1565, 2)

In [20]:
BusTrans =  @linq Bus |>
            where(:kV .> 38) |>
            select(:Bus,:kV);
BusNo=size(BusTrans,1)

545

In [2]:
BranchFrame = DataFrame(XLSX.readtable("All-Ireland System- WinterPeak 2026 - PSSE Data.xlsx", "AC_Line", infer_eltypes=true)... )
BranchFrame

,From,From Bus Name,To,To Bus Name,Id,Name,Term Node Num (From)
,Int64,String,Int64,String,Int64,Any,Any
1,11,FMN LOOP 38.000,42404,NORTH_WALL 38.000,1,missing,missing
2,12,BEDFORD ROW 38.000,25404,FRANCIS ST 38.000,1,missing,missing
3,12,BEDFORD ROW 38.000,25404,FRANCIS ST 38.000,3,missing,missing
4,12,BEDFORD ROW 38.000,46504,RINGSEND 38.000,1,missing,missing
5,12,BEDFORD ROW 38.000,46504,RINGSEND 38.000,2,missing,missing
6,1021,ARDNACRUSHA 110.00,2121,DRUMLINE 110.00,1,missing,missing
7,1021,ARDNACRUSHA 110.00,2361,ENNIS 110.00,1,missing,missing
8,1021,ARDNACRUSHA 110.00,3541,LIMERICK 110.00,1,missing,missing
9,1021,ARDNACRUSHA 110.00,10210,ARDNACRUSHA 110.00,1,missing,missing


In [33]:
BranchDict= Dict{Tuple{Float64, Float64, String}, Float64}();  

In [4]:
size(BranchFrame,1)

1187

In [32]:
filter(row -> BranchFrame ∈ [1,2,3], BranchFrame )

,From,From Bus Name,To,To Bus Name,Id,Name,Term Node Num (From),Term Node Name (From)
,Int64,String,Int64,String,Int64,Any,Any,Any


In [92]:
r = BranchFrame.R;
x = BranchFrame.X;
r[1]

0.0036

In [91]:
BranchDict =nothing
for Par in [Symbol(R),Symbol(X)]
    for i in 1:size(BranchFrame,1)
        BranchDict[(BranchFrame.From[i],BranchFrame.To[i], BranchFrame.Par[i])] = Par[i]
    end
end

MethodError: MethodError: no method matching getindex(::Symbol, ::Int64)

In [55]:
string(R)

"Any[0.0036, 0.00534, 0.00594, 0.02132, 0.01937, 0.0286, 0.0208, 0.0071, 0, 0, 0.0049, 0, 0.0001, 0.003, 0.002627, 0.0001, 0, 0.0002, 0.003, 0.0004, 0, 0.0125, 0.0594, 0.0718, 0.0039, 0.0018, 0.0091, 0.0031, 0.0102, 0.0207, 0.0035, 0.0062, 0.0046, 0.0032, 0, 0.0057, 0.054, 0.0135, 0, 0, 0.001, 0.0207, 0.0173, 0.0006, 0.0006, 0.001, 0.0666, 0.0193, 0.0412, 0.0117, 0.0148, 0.0047, 0.0026, 0.002094, 0.0001, 0, 0.002428, 0.002, 0.0389, 0.0138, 0.0111, 0.0151, 0.0124, 0.0106, 0.0126, 0.0426, 0.0148, 0.0021, 0.0006, 0.0533, 0.0242, 0.0121, 0.0073, 0.0312, 0.0071, 0, 0.0027, 0.0021, 0.0533, 0.017, 0, 0, 0.0047, 0.0047, 0.002, 0.003, 0.0139, 0.0176, 0.004, 0.0403, 0.0413, 0, 0.0039, 0.0006, 0.0035, 0.0004, 0.000354, 0.000354, 0.0004, 0.0078, 0.0038, 0.0094, 0.0047, 0.0047, 0.0179, 0.008, 0.0544, 0.0024, 0.0025, 0.0188, 0.022, 0.0036, 0, 0.0062, 0.0128, 0, 0.004, 0.0094, 0.0046, 0.0197, 0.0086, 0.0144, 0.0745, 0.0336, 0.0215, 0.0176, 0.0176, 0.0248, 0, 0, 0, 0.0778, 0.0103, 0.0103, 0.0046, 0.01

In [14]:
BranchDict[(1121,1541)]

0.0952

In [22]:
Gw = SimpleWeightedGraph(BusNo)

{545, 0} undirected simple Int64 graph with Float64 weights

In [64]:
BranchInter1 = Branch[findall(in(BusTrans.Bus), Branch.From),:];
BranchInter1

,From,From Bus Name,To,To Bus Name,Id,Name,Term Node Num (From)
,Int64,String,Int64,String,Int64,Any,Any
1,1021,ARDNACRUSHA 110.00,2121,DRUMLINE 110.00,1,missing,missing
2,1021,ARDNACRUSHA 110.00,2361,ENNIS 110.00,1,missing,missing
3,1021,ARDNACRUSHA 110.00,3541,LIMERICK 110.00,1,missing,missing
4,1021,ARDNACRUSHA 110.00,10210,ARDNACRUSHA 110.00,1,missing,missing
5,1021,ARDNACRUSHA 110.00,10261,ARDNA_CAP 110.00,1,missing,missing
6,1041,AGHADA 110.00,5481,WHITEGATE 110.00,1,missing,missing
7,1042,AGHADA 220.00,1052,AGHADA_B 220.00,1,missing,missing
8,1042,AGHADA 220.00,1072,AGHT2102 220.00,1,missing,missing
9,1042,AGHADA 220.00,3202,KNOCKRAHA 220.00,1,missing,missing


In [66]:
BranchInter2 = Branch[findall(in(BusTrans.Bus), Branch.From),:];
BranchInter2 

,From,From Bus Name,To,To Bus Name,Id,Name,Term Node Num (From)
,Int64,String,Int64,String,Int64,Any,Any
1,1021,ARDNACRUSHA 110.00,2121,DRUMLINE 110.00,1,missing,missing
2,1021,ARDNACRUSHA 110.00,2361,ENNIS 110.00,1,missing,missing
3,1021,ARDNACRUSHA 110.00,3541,LIMERICK 110.00,1,missing,missing
4,1021,ARDNACRUSHA 110.00,10210,ARDNACRUSHA 110.00,1,missing,missing
5,1021,ARDNACRUSHA 110.00,10261,ARDNA_CAP 110.00,1,missing,missing
6,1041,AGHADA 110.00,5481,WHITEGATE 110.00,1,missing,missing
7,1042,AGHADA 220.00,1052,AGHADA_B 220.00,1,missing,missing
8,1042,AGHADA 220.00,1072,AGHT2102 220.00,1,missing,missing
9,1042,AGHADA 220.00,3202,KNOCKRAHA 220.00,1,missing,missing


In [69]:
Comparison = BranchInter1.From .== BranchInter2.From;
unique(Comparison)

1-element Array{Bool,1}:
 1

### Define the bus set

In [32]:
Bus = max(length(unique(Branch.From)),length(unique(Branch.To)))

822

### Defining Susceptance Matrix (Laplacian)
##### Note: only Susceptance Matirx is defined for Linear power flow

In [33]:
Gw = SimpleWeightedGraph(BusNo)

for i = 1:size(Branch,1)
    add_edge!(Gw,Branch.From[i],Branch.To[i], 1/Branch.R[i])
end

In [34]:
Aw = Array(weights(Gw))

822×822 Array{Float64,2}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.

In [ ]:
Aw = Array(weights(Gw))
Lw = Diagonal(sum(Aw,dims=1)[:])-Aw
B = Lw;

# Generator Location Data

In [5]:
#GenData = DataFrame(a = [0.001562, 0.00194, 0.00482], 
#                b = [7.92, 7.85, 7.97],
#                c = [561, 310, 78],
#                Pmax = [600, 400, 200],
#                Pmin = [150, 100, 50],
#                RampUp = [30,20,30],
#                RampDown = [30,20,30],
#                MinUp = [2,3,2],
#                MinDown = [2,3,2]);

In [6]:
GenData = DataFrame(XLSX.readtable("3-Bus.xlsx", "GenData", infer_eltypes=true)... )

,Name,Type,a,b,c,Pmax,Pmin,RampUp,RampDown,MinUp
,Int64,String,Float64,Float64,Int64,Int64,Int64,Int64,Int64,Int64
1,1,Unknown,0.001562,7.92,561,600,150,30,30,2
2,2,Unknown,0.00194,7.85,310,400,100,20,20,3
3,3,Unknown,0.00482,7.97,78,200,50,30,30,2


In [7]:
PVar = DataFrame(Bus = [1, 2, 3], Gen = [1,1,1])

Bus = max(maximum(Branch.From),maximum(Branch.To))
Genw = SimpleDiGraph(Bus)

for i = 1:size(PVar,1)
    add_edge!(Genw,PVar.Bus[i],PVar.Gen[i] )
    println(PVar.Bus[i],PVar.Gen[i])
end
AGenw = Array(adjacency_matrix(Genw))

11
21
31


3×3 Array{Int64,2}:
 1  0  0
 1  0  0
 1  0  0

### Define Generator Set

In [8]:
Gen = size(GenData, 1)

3